# Financial Markets Prices
---
### Autor: Diego Alegría   2023-02-24 v3
#### Basado en código de J Gonzalez
---
Código para obtener valores de ~ 257 precios relevantes para diversos reportes.
Utilizado todos los viernes después de que cierre la bolsa. (4 pm en Nueva York, 2 o 3 en Guate)

Usos primarios son un reporte semanal de Promerica el viernes, y para reporte de Commodities el lunes.

- Referencias para arreglo:
https://github.com/JECSand/yahoofinancials/blob/master/yahoofinancials/__init__.py
https://github.com/JECSand/yahoofinancials/issues/105
https://github.com/JECSand/yahoofinancials/commit/62119be23fe81f89fd5fb1bd450fc2088667c8df

---
### Instrucciones de actualizacion.

Solamente actualizar yahoofinancials con pip, El autor JECSand implementó los arreglos propuestos por GitHub.

- 257 precios son actualizados por medio de yahoofinancials.
- 10 variables son actualizadas manualmente, utilizando el excel Markets_Temporal.xlsx ubicado en DB\Weekly\Viernes Temporales\

Hay una seccion en este codigo al final que realiza el procesamiento para subir estas ultimas variables sin tener que abrir SQL.

# Importación de liberías

In [3]:
# !pip install yahoofinancials
# !pip install pyodbc

In [4]:
# import investpy
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
from yahoofinancials import YahooFinancials
from datetime import*
import urllib
import numpy as np
import time
import pyodbc
import traceback
import requests as requests
import numpy as np
import sys
from sqlalchemy import*

from pandas import Timestamp
style.use('ggplot')

#Fecha de actual: (Lugar para Hardcoded abajo)
fecha_inicial=datetime.today().strftime('%Y-%m-%d')
fecha_final=(datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

# Funciones par SQL

Información importante sobre esta celda

- El código de lectura trae de regreso 3 querys que se utilizan en este notebook
  - El primer query trae de regreso la última fecha
  - El segundo trae una serie de fechas que son las necesarias obtener en la base de datos para homologar si se agrega un índice nuevo
  - El tercero es un sample para crear el frame que se subirá a la base de datos

In [5]:
driver= '{ODBC Driver 17 for SQL Server}'
DATABASE_CONFIG = {
    'server': 'datacabisvr2.database.windows.net',
    'database': 'CABI_DB',
    'username': 'eelvira',
    'password': 'Cabi2023!'
}

server= 'datacabisvr2.database.windows.net'
database='CABI_DB'
username= 'dalegria'
password= 'Cabi2019'

connection_string = 'DRIVER=' + driver + \
                     ';SERVER=' + server + \
                    ';PORT=1435' + \
                    ';DATABASE=' + database + \
                    ';UID=' + username + \
                    ';PWD=' + password

def write2db(df,table, fast = True):
    # prepare SQLAlchemy engine
    # Doc: https://docs.sqlalchemy.org/en/latest/dialects/mssql.html#module-sqlalchemy.dialects.mssql.pyodbc
    # same connection string as before
    params = urllib.parse.quote_plus(connection_string)
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params, fast_executemany=fast)
    flag=1
    # df columns names need to match the table column names in DB

    # check #rows in table before write operation
    sql_conn = pyodbc.connect(connection_string)
    cursor = sql_conn.cursor()
    query = '''
                SELECT count(*)
                FROM [dbo].['''+table+"]"

    try:
        cursor.execute(query)
        rows_before = cursor.fetchone()
        flag=1
    except:
        print('No table, creating a new table')
        flag=0


    # write results to Azure DB table
    df.to_sql(table, con = engine, if_exists = 'append', index = False)
    if(flag==1):
        # check #rows in table after write operation
        cursor.execute(query)
        rows_after = cursor.fetchone()
        print(rows_after)
        print("Inserted {} rows into table {}".format(rows_after[0]-rows_before[0], table))

    else:
        print('New table created')
    return


server= 'datacabisvr2.database.windows.net'
database='CABI_DB'
username= 'eelvira'
password= 'Cabi2023!'
driver= '{ODBC Driver 17 for SQL Server}'
connection_string = 'DRIVER=' + driver + \
                     ';SERVER=' + server + \
                    ';PORT=1435' + \
                    ';DATABASE=' + database + \
                    ';UID=' + username + \
                    ';PWD=' + password

def read_data_to_df(query):
    try:
        print("    \u2713 connecting db: " + database)
        sql_conn = pyodbc.connect(connection_string)
        # execute query and save data in pandas df
        df = pd.read_sql(query, sql_conn)
        return df
    except Exception as error:
        print("    \u2717 accesing {} has error".format(database))
        print("    \u2717 error message: {}".format(error))
        # I found that traceback prints much more detailed error message
        traceback.print_exc()

if __name__ == "__main__":

    # Query para tomar el último dato del dji
    query_fecha = '''
            SELECT        MAX(DatoFecha)
            FROM            dbo.TablaDatos
            WHERE        (IdIndiceEspecifico = 2988)


                '''

    query_fechas = '''
            SELECT        DatoFecha
            FROM            dbo.TablaDatos
            WHERE        (IdIndiceEspecifico = 2986)


                '''

    querydf = '''
SELECT
      TablaDatos.[IdIndiceEspecifico]
      ,[DatoFecha]
      /*,[D_Open]
      ,[D_High]
      ,[D_Low]*/
      ,[D_Close]

      ,Indice_Especifico.Nombre
      /*,Indice_Especifico.Descripcion
      ,Indice_Especifico.IdIndicador*/

  FROM [dbo].[TablaDatos]
  INNER JOIN Indice_Especifico
  ON TablaDatos.IdIndiceEspecifico=Indice_Especifico.IdIndiceEspecifico
  WHERE Indice_Especifico.IdIndicador=21
  OR Indice_Especifico.IdIndicador=22
  OR Indice_Especifico.IdIndicador=30
  OR Indice_Especifico.IdIndicador=37
            '''

## Estas tres consultas sirven para definir varios parámetros de los datos a subir.
# esto incluye fechas,
last_date = read_data_to_df(query_fecha)


fechas = read_data_to_df(query_fechas)

commdf=read_data_to_df(querydf)

#last_date_value=last_date.loc[0][0]
#last_date_value=datetime.strftime(last_date_value,'%Y-%m-%d')
#fechas.DatoFecha=pd.to_datetime(fechas.DatoFecha)
#last_date_value

    ✓ connecting db: CABI_DB


C:\Users\lijv1\AppData\Local\Temp\ipykernel_18388\3026994659.py:77: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, sql_conn)


    ✓ connecting db: CABI_DB
    ✓ connecting db: CABI_DB


In [6]:
last_date_value = '2025-7-11'

---
# Lectura de Tickets ()

Si se agrega un nuevo índice se debe agregar a esta base de datos

In [7]:
#Lectura de indices utilizados para cada variable charl
# En Source se encuentra de donde se puede obtener
try:
    df=pd.read_excel(r'CARGA NUEVOS ETFs SQL 2020.xlsx',
#     df=pd.read_excel(r'C:\Users\lijv1.LINDA_HP\CABI\Data Bases - Documents\Weekly\CARGA NUEVOS ETFs SQL 2020.xlsx',
                  sheet_name='TickersDownload')
except:
    print('Falla encontrando archivo. Intentando por D. Alegria: ')
    df=pd.read_excel(r'C:\Users\aleja\CABI\Data Bases - Documentos\Weekly\CARGA NUEVOS ETFs SQL 2020.xlsx',
                  sheet_name='TickersDownload',usecols=np.arange(0,7))
display(df)

,IdTD,IdTicker,Source,Descripcion,IdIndiceEspecifico
0,$BCOM(D),AW=F,Yahoo,Blmbg Commodity Index,3006
1,$BCOMEN(D),$BCOMEN(D),Pendiente,Blmbg Energy Sub,3775
2,$BCOMGR(D),$BCOMGR(D),Investing,Blmbg Grains Subindex,3007
3,$BCOMIN(D),$BCOMIN(D),Investing,Blmbg Industrial Mtls Sub,3008
4,$BCOMPR(D),$BCOMPR(D),Investing,Blmbg Precious Metals Sub,3009
...,...,...,...,...,...
261,XRT(HB),XRT,Yahoo,SPDR S&P Retail,2964
262,ZO=F,ZO=F,Yahoo,Oats Continuous Contract [Sep21],3022
263,ZR=F,ZR=F,Yahoo,Rough Rice,3998
264,DC=F,DC=F,Yahoo,Class III Milk,4020


## Filtro Manual de variables

In [8]:
from datetime import date

def dia_y_mes_hoy():
    hoy = date.today()
    return hoy.day, hoy.month  # (día, mes)

d, m = dia_y_mes_hoy()
print(d, m)

27 10


In [9]:
# #Filtro de variables manual:

lista_manual = ['ERUS', 'RSX']

# #Fecha Manual (Cambiar a automático para tomar la fecha de hoy):
fecha_inicial=dt.datetime(2025,m,d)
fecha_final=dt.datetime(2025,12,31)

# Scrapper para Yahoo para Data TD

> Se recoge del DF anterior llamado donde están los indicadores todos aquellos donde su fuente es yahoo.com, se utiliza la librería de pandas Webreader para tomar la información.

> Debajo se crea un df_y que será el que va a contener la última lectura de cada mercado que se toma de yahoo finance y se le va agregando información que posteriormente será homologada a el formato que está en tabladatos

> Tiene un modo de fallo en donde se utiliza una vez falla la primera recolección de datos, utiliza una segunda oportunidad puesto que en el 99% de los casos itera dos veces y jala bien el indicador, si se quiere agregar más protección solo encerrar en un ciclo while, evitar usarlos por eficiencia.

In [10]:
## CELL DE INICIALIZACION Y DEFINICIONES
from tqdm.notebook import tqdm, trange     #Comentar si no se quiere utilizar progress bar
# from tqdm import tqdm, trange            #Progress bar alternativo

#Definicion de variables obtenidas desde yahoo
y=df[df['Source']=='Yahoo']
y.reset_index(inplace=True,drop=True)

#Construccion de dataframe contenedor
df_y=pd.DataFrame(columns=['IdIndiceEspecifico','Date', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'
                           ,'Descripcion', 'IdTD','IdTicker','Source'])



import datetime as dt
import yfinance as yf



#Definicion de funciones para extraccion de datos del paquete bajado:
def unpack(initial, ind):
    precios = initial

    #print("PRECIOS", precios.columns)

    #display(precios)

#     display(precios.Date)
    #Extraccion de valores de precios:
    last = pd.DataFrame({'DatoFecha':[precios['Date'][0]]})
    last['D_Open'] = precios['Open']
    last['D_High'] = precios['High']
    last['D_Low'] = precios['Low']
    last['D_Close'] = precios['Close']
    last['D_Volume'] = precios['Volume']
    last['D_OpenInterest'] = precios['Adj Close']
#     if ('Adj Close', 'AW=F') in precios.columns:
#         last['D_OpenInterest'] = precios[('Adj Close', 'AW=F')]
#     else:
#         last['D_OpenInterest'] = 0  # O cualquier otro valor predeterminado
    #Extraccion de valores de formato:
    last['IdIndiceEspecifico']=ind.IdIndiceEspecifico
    last['Descripcion']=ind.Descripcion
    last['IdTD']=ind.IdTD
    last['IdTicker']=ind.IdTicker
    last['Source']=ind.Source
    return last


def downloading_yahoo(indice,fecha_i,fecha_f):
    ind = indice.IdTicker
    yahoo_financials = yf.download(ind, fecha_i, fecha_f, auto_adjust=False)
    if yahoo_financials.empty:
        raise ValueError("El DataFrame 'yahoo_financials' está vacío. Verifica la descarga de datos.")

    yahoo_financials.reset_index(inplace=True,drop=False)

    display(yahoo_financials)

    return unpack(yahoo_financials,indice)

# if 'lista_manual' in locals:
#     y_descarga=y[y.IdTicker.isin(lista)].copy()
#     y_descarga.reset_index(inplace=True)
# else:
y_descarga = y.copy()

# Quick test
idx=y.loc[0]
display(idx.IdTicker)
last = downloading_yahoo(idx, fecha_inicial, fecha_final)
last

'AW=F'

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AW=F']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


ValueError: El DataFrame 'yahoo_financials' está vacío. Verifica la descarga de datos.

In [ ]:
y_descarga

In [ ]:
## DESCARGA COMPLETA de YAHOO
missing = [] #Indices de faltantes
a_juntar = [] #Variable para juntar dataframes
espera = 1 # seg
retries = 3

for i in trange(len(y_descarga)):     #Reemplazar con range si no se quiere utilizar progress bar
# for i in range(0,1):
    #Procesamiento original
    idx=y_descarga.loc[i]
#     display(idx.IdTicker)
    n=0
    #Intentos de descarga:
    for n in range(retries):
        try:     #1er Intento
            #Usando funciones definidas
            last = downloading_yahoo(idx, fecha_inicial, fecha_final)
            #Agregar y esperar para debugging
            a_juntar.append(last)
            time.sleep(espera)
            break #EXITO, pasar a siguiente indice.
        except:   #Segundo intento en caso de falla
            print('Fallo con Ticker')
            time.sleep(espera)
            print('Entrando en modo de fallo')
            if n == retries-1:
                display(idx.IdTicker)
                missing.append(idx.IdTicker)

#Construir data frame final.
df_y = pd.concat(a_juntar)
display(missing)
df_y

## Filtro específico para ocasiones especiales

In [ ]:
# df_y=df_y.loc[df_y.DatoFecha==df_y.DatoFecha[0].tolist()[0],:].copy()

In [ ]:
import pandas as pd
from IPython.display import display

# Ajustar la configuración de visualización de pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# ...

solicitados = len(y_descarga)
recibidos = len(df_y)
print('Del total de {} datos solicitados, se recibieron {} datos.'.format(solicitados, recibidos))

df_y_backup = df_y.copy()

print('Indices faltantes:')
display(missing)

print('DataFrame completo:')
display(df_y)


In [12]:
#Preparacion para correr esta seccion de nuevo.
lista_manual = missing

#### Se quiere recibir 257

## En caso de error... buscador de tickets para manual o intentar de nuevo.

Por si no retorna uno de los datos por un fallo inminente utilizar esto para buscar el índice que falló y luego ingresarlo manual en la base de datos

#### Datos no publicados:
- AW=F Para 2022-12-23 !


In [13]:
# missing=['EURUSD=X','GBPUSD=X','MXN=X']
df[df.IdTicker.isin(missing)]

,IdTD,IdTicker,Source,Descripcion,IdIndiceEspecifico
63,EGPT(HB),EGPT,Yahoo,VanEck Vctrs Egypt Idx,251
82,EWGS(HB),EWGS,Yahoo,iShs MSCI Germany Sm Cap,234
117,GRES(HB),GRES,Yahoo,IQ Global Resources ETF,2912
167,JJU,JJU,Yahoo,iPath Series B Bloomberg Aluminum Subindex Tot...,3780
192,PEK,GLCN,Yahoo,VanEck Vectors China Growth Leaders ETF,3774


In [ ]:
#Ejemplo para buscar faltantes:
#query= '''
#SELECT TOP(50) * FROM dbo.TablaDatos Where IdindiceEspecifico = 251 Order By DatoFecha Desc
#'''
#dfOJ = read_data_to_df(query)
#dfOJ[:]

## Unificación final y empacado de datos para base de datos

Aquí se unifican (Cuando está disponible) los índices de Yahoo e Investing y se empacan en el formato de TablaDatos

In [14]:
df_final=df_y.copy()
df_final.to_excel('FinancialMarkets.xlsx',index=False)
# display(df_final)

#Ordenar
df_final=df_final[['IdIndiceEspecifico',
     'DatoFecha',
     'D_Open',
     'D_High',
     'D_Low',
     'D_Close',
     'D_Volume',
     'D_OpenInterest']]
#Renombrar
df_final.columns=['IdIndiceEspecifico',
     'DatoFecha',
     'D_Open',
     'D_High',
     'D_Low',
     'D_Close',
     'D_Volume',
     'D_OpenInterest']
df_final.reset_index(inplace=True,drop=True)
df_f=df_final.copy() #Copia a final
df_f.reset_index(inplace=True,drop=True)


df_f.DatoFecha=df_final.loc[1,'DatoFecha']


In [ ]:
df_f

## Retry para agregar faltantes

In [ ]:
for item in ['ERUS', 'RSX']:
    try:
        lista_manual.remove(item)
    except ValueError:
        pass

lista_manual


ValueError: list.remove(x): x not in list

In [17]:
# for i in range(0,1):
    #Procesamiento original
idx=lista_manual[-1]
#     display(idx.IdTicker)
n=0
#Intentos de descarga:
for n in range(retries):
    try:     #1er Intento
        #Usando funciones definidas
        last = downloading_yahoo(idx, fecha_inicial, fecha_final)
        #Agregar y esperar para debugging
        a_juntar.append(last)
        time.sleep(espera)
        break #EXITO, pasar a siguiente indice.
    except:   #Segundo intento en caso de falla
        print('Fallo con Ticker')
        time.sleep(espera)
        print('Entrando en modo de fallo')
#         if n == retries-1:
#             display(idx.IdTicker)
#             missing.append(idx.IdTicker)

Fallo con Ticker
Entrando en modo de fallo
Fallo con Ticker
Entrando en modo de fallo
Fallo con Ticker
Entrando en modo de fallo


## Escritura a la base de datos

Se escribe a TablaDatos y TablaDatos_BU hasta que se complete la migración

In [ ]:
df_f

In [ ]:
#write2db(df_f,'TablaDatos')
df_f.to_excel('FinancialMarketsBU.xlsx',index=False)